In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import codecs
import time

In [2]:
home = 'http://www.chinavitae.com/'
location = 'http://www.chinavitae.com/location/'
loc_start_html = requests.get(location)
loc_soup = BeautifulSoup(loc_start_html.text, 'lxml')

In [3]:
""" 
    First layer: 201 different locations. 'http://www.chinavitae.com/location/xxx'
    Find <a> and 'href' attributes 
    Output: a_tag_dic = {link:location}
    A dictionary of 201 elements, each of them is a location
"""
def find_a_tag(soup, url):
    a_tag = soup.find_all('a')
    a_tag_dic = {}
    for a in a_tag:
        if a.has_attr('href'):
            attr = a['href']
            attr_search = re.compile('/location/.+')
            attr_str = re.match(attr_search, attr)
            if attr_str and str(attr_str.group()).find('&show') == -1:
                a_tag_dic[home+attr_str.group()[1:]] = a.text
    return a_tag_dic

In [4]:
""" Function call. Get location layer 1 """
loc_layer1_locations = find_a_tag(loc_soup, loc_start_html.url)

In [5]:
# print len(loc_layer1_locations)
# print loc_layer1_locations

In [6]:
"""
    Second layer: 201 locations and their sub-locations
    'http://www.chinavitae.com/institution/xxx' and 'http://www.chinavitae.com/location/xxx.xxx'
    
    List of size 201, each element belongs to a location.
    Each element is a dictionary of multiple {url: location_name}
"""
start_time = time.time() # Timer
loc_layer1_counter = 0 # Counter
loc_layer2_subLocations = []
for url, val in loc_layer1_locations.iteritems():
    url_html = requests.get(url)
    url_soup = BeautifulSoup(url_html.text, 'lxml')
    loc_layer2_subLocations.append(find_a_tag(url_soup, url_html.url))
    loc_layer1_counter += 1 # Counter
#     break
print("--- %s seconds ---" % (time.time() - start_time)) # Print running time

--- 101.794301987 seconds ---


In [7]:
# print len(loc_layer2_subLocations)
# print loc_layer2_subLocations[1]

In [8]:
"""
    Convert [{}] to {}
    Store all [{url:name}] in to dictionary 
"""
loc_final_counter = 0
loc_final = {} # Final location dictionary
for loc in loc_layer2_subLocations:
    for key, val in loc.iteritems():
        loc_final_counter += 1
        loc_final[key] = val        

In [9]:
# print loc_final_counter
# print loc_final

In [10]:
"""
    Get category and id from url
    url_cat: category
    url_id: id
"""
def get_category_and_id_from_url(url):
    url_cat = ''
    url_id = ''
#     base = 'http://www.chinavitae.com/location/'
    piece = url.split('/')

    if piece[-1][0].isdigit():
        url_id = piece[-1]
        url_cat = piece[-2]
    else:
        url_id = url_cat = piece[-1]
    return [url_cat, url_id]

In [11]:
"""
    Write location into files
"""
# everything = layer3_sub_subCategories
myfile = codecs.open('loc-list.txt', 'w', 'utf-8')
i = 0
for url, name in loc_final.iteritems():
    [tmp_cat, tmp_id] = get_category_and_id_from_url(url)
    myfile.write(tmp_cat + ', ' + tmp_id + ', ' + name + '\n')
    i += 1
myfile.close()
print i

2585
